In [22]:
import tensorflow as tf
import numpy as np

In [18]:
def residual_connection(inputs,n_channels):
    x = tf.keras.layers.Conv2D(n_channels,(3,3),padding='same')(inputs)
    x = tf.keras.layers.Conv2D(n_channels,(3,3),padding='same')(x)
    return x+inputs

    

In [39]:
def upsize(x,upsize=2):
    x = tf.keras.layers.Conv2D(x.shape[-1]*upsize,(3,3),padding='same')(x)
    return x

In [44]:
class MyModel(tf.keras.Model):
    def __init__(self,n_classes):
        super(MyModel,self).__init__()
        self.Conv2D_7 = tf.keras.layers.Conv2D(64,(7,7),strides=2,padding='same')
        self.maxpool = tf.keras.layers.MaxPool2D((3,3),strides=2,padding='same')
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(n_classes)
    
    def call(self,inputs):
        x = self.Conv2D_7(inputs)
        x = self.maxpool(x)
        
        for i in range(3):
            x = residual_connection(x,64)
        
        print(x.shape) # (1, 56, 56, 64)
        x = self.maxpool(x)
        x = upsize(x)
        print(x.shape) # (1, 28, 28, 128)
        
        for i in range(4):
            x = residual_connection(x,128)
        
        print(x.shape) # (1, 28, 28, 128)
        x = self.maxpool(x)
        x = upsize(x)
        print(x.shape) # (1, 14, 14, 256)
        
        for i in range(5):
            x = residual_connection(x,256)
        
        print(x.shape) # (1, 14, 14, 256)
        x = self.maxpool(x)
        x = upsize(x) 
        print(x.shape) # (1, 7, 7, 512)
        
        for i in range(3):
            x = residual_connection(x,512)
            
        x = self.avgpool(x)
        x = self.fc(x)
        return x
        

In [45]:
model = MyModel(1000)

In [46]:
x = np.random.randn(1,224,224,3)

In [47]:
print(model(x).shape) # (1, 1000)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(1, 56, 56, 64)
(1, 28, 28, 128)
(1, 28, 28, 128)
(1, 14, 14, 256)
(1, 14, 14, 256)
(1, 7, 7, 512)
(1, 1000)


In [48]:
model.summary()

Model: "my_model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_123 (Conv2D)          multiple                  9472      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 multiple                  0         
_________________________________________________________________
global_average_pooling2d_2 ( multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  513000    
Total params: 522,472
Trainable params: 522,472
Non-trainable params: 0
_________________________________________________________________
